<a href="https://colab.research.google.com/github/MezaMauricio/Portfolio/blob/main/C08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Caso Churn

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
#import plotly.express as px
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('https://raw.githubusercontent.com/dfbustosus/Curso_DS_para_todos/main/Data/train_churn.csv')
df = train_df.copy()

print('Shape {}'.format(df.shape))
print('Filas:{}'.format(df.shape[0]))
print('Columnas:{}'.format(df.shape[1]))
df.info()

Shape (4250, 20)
Filas:4250
Columnas:20
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4250 entries, 0 to 4249
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   state                          4250 non-null   object 
 1   account_length                 4250 non-null   int64  
 2   area_code                      4250 non-null   object 
 3   international_plan             4250 non-null   object 
 4   voice_mail_plan                4250 non-null   object 
 5   number_vmail_messages          4250 non-null   int64  
 6   total_day_minutes              4250 non-null   float64
 7   total_day_calls                4250 non-null   int64  
 8   total_day_charge               4250 non-null   float64
 9   total_eve_minutes              4250 non-null   float64
 10  total_eve_calls                4250 non-null   int64  
 11  total_eve_charge               4250 non-null   float64
 12  total_ni

In [3]:
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


Objetivo: predecir si un cliente cambiará de proveedor de telecomunicaciones, algo que se conoce como "churning".

El conjunto de datos de entrenamiento contiene 4250 muestras. Cada muestra contiene 19 variables y 1 variable booleana "churn" que indica la clase de la muestra. Las 19 características de entrada y 1 variable objetivo son:

"state", string. 2-letter code of the US state of customer residence

"account_length", numerical. Number of months the customer has been with the current telco provider

"area_code", string="area_code_AAA" where AAA = 3 digit area code.

"international_plan", (yes/no). The customer has international plan.

"voice_mail_plan", (yes/no). The customer has voice mail plan.

"number_vmail_messages", numerical. Number of voice-mail messages.

"total_day_minutes", numerical. Total minutes of day calls.

"total_day_calls", numerical. Total minutes of day calls.

"total_day_charge", numerical. Total charge of day calls.

"total_eve_minutes"-, numerical. Total minutes of evening calls.

"total_eve_calls"-, numerical. Total number of evening calls.

"total_eve_charge"-, numerical. Total charge of evening calls.

"total_night_minutes"-, numerical. Total minutes of night calls.

"total_night_calls"-, numerical. Total number of night calls.

"total_night_charge"-, numerical. Total charge of night calls.

"total_intl_minutes"-, numerical. Total minutes of international calls.

"total_intl_calls"- numerical. Total number of international calls.

"total_intl_charge"-, numerical. Total charge of international calls

"number_customer_service_calls"-, numerical. Number of calls to customer service

"churn"-, (yes/no). Customer churn - target variable.

**Tipos de data**

✈ ⚗ 🥲 ⚾

Categoricas

'state' - Nominal

'area_code' - Nominal

'international_plan' - Binary

'voice_mail_plan' - Binary



**Continuas**



'account_length'

'number_vmail_messages'

'total_day_minutes'

'total_day_calls'

'total_day_charge'

'total_eve_minutes'

'total_eve_calls'

'total_eve_charge'

'total_night_minutes'

'total_night_calls'

'total_night_charge'

'total_intl_minutes'

'total_intl_calls'

'total_intl_charge'

'number_customer_service_calls'


In [4]:
# Propiedades para las figuras
axtitle_dict = {'family': 'serif','color':  'darkred','weight': 'bold','size': 15}
axlab_dict = {'family': 'serif', 'color': 'black','size': 14}

In [5]:
# Miremos balance de categoria
a = round(df['churn'].value_counts()/len(df['churn']),2)
print(a) 

no     0.86
yes    0.14
Name: churn, dtype: float64


# Data Wrangling

## nulos + Duplicados